In [0]:
from sparkmeasure import StageMetrics
#from pyspark.sql.functions import trim
from pyspark.sql.window import Window
from pyspark.sql import functions as F
stagemetrics = StageMetrics(spark)

In [0]:
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"

In [0]:
import re

def trim(df):
  return df.toDF(*[re.sub('^[ \t]+|[ \t]+$', '', x) for x in df.columns])

In [0]:
def airlines_rankings(spark_session, flights_path, airlines_path, aircrafts_path, country):
    flights = spark_session.read.csv(
        flights_path,
        inferSchema=True,
        header=True
    )
    flights = trim(flights).select(F.col('carrier_code'), F.col('tail_number'))

    airlines = (
        spark_session.read.csv(
            airlines_path,
            inferSchema=True,
            header=True
        ).filter(F.col('country') == country).drop('country')
    )

    aircrafts = (
        spark_session.read.csv(
            aircrafts_path,
            inferSchema=True,
            header=True
        ).select([
            'tailnum',
            'manufacturer',
            'model'
        ])
    )

    aircrafts = aircrafts.withColumn(
        'model', F.regexp_replace(F.col('model'), '-', '')
    ).withColumn('model', F.regexp_extract(F.col('model'), '.+?(?=\d)\d{2}', 0))


    airlines_aircrafts = (
        airlines.join(
            flights,
            on='carrier_code',
            how='left_outer'
        )
        .filter(airlines.name.isNotNull())
        .join(
            aircrafts,
            flights.tail_number == aircrafts.tailnum,
            'inner'
        ).dropna()
        .groupBy('name', 'manufacturer', 'model')
        .count()
    )

    airline_ranked_models = airlines_aircrafts.withColumn(
        "rank", F.rank().over(Window.partitionBy('name').orderBy(F.desc('count')))
    ).filter(F.col('rank') <= 5)

    result = (
        airline_ranked_models.withColumn('aircraft', 
            F.concat(
                F.col('manufacturer'), 
                F.lit(" "), F.col('model')
            )
        ).orderBy('rank', descending=False)
    ).groupBy('name').agg(
        F.collect_list(F.struct('aircraft')).alias('aircrafts_per_airline')
    ).orderBy('name')

    output = ""
    for (name, aircrafts) in result.collect():
        output += f"{name} \t["
        for i in range(0, len(aircrafts)-1):
            output += "%s, " % aircrafts[i]
        output += "%s]\n" % aircrafts[-1]

    print(output)

In [0]:
stagemetrics.begin()
airlines_rankings(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", 
                         f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                         f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717, BOEING 737]
Alaska Airlines Inc. [BOEING 737, MCDONNELL DOUGLAS DC98, MCDONNELL DOUGLAS AIRCRAFT CO MD 83]
American Airlines Inc. [MCDONNELL DOUGLAS DC98, AIRBUS INDUSTRIE A31, BOEING 757, AIRBUS INDUSTRIE A32, BOEING 767]
American Eagle Airlines Inc. [EMBRAER EMB14, EMBRAER EMB13, BOMBARDIER INC CL60, SAAB-SCANIA SAAB 34]
Atlantic Southeast Airlines [BOMBARDIER INC CL60, EMBRAER EMB14, CANADAIR CL60, EMBRAER EMB13, AEROSPATIALE/ALENIA ATR72]
Comair [BOMBARDIER INC CL60, CANADAIR CL60, PIPER PA28]
Continental Air Lines Inc. [BOEING 737, BOEING 757, BOEING 767, BOEING 777]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD88, BOEING 757, BOEING 737, BOEING 717, BOEING 767]
Frontier Airlines Inc. [AIRBUS A31, AIRBUS INDUSTRIE A31, AIRBUS A32]
Hawaiian Airlines Inc. [BOEING 717, BOEING 767]
Independence Air [BOMBARDIER INC CL60]
JetBlue Airways [AIRBUS A32, EMBRAER ERJ 19, AIRBUS INDUSTRIE A32]
JetSuite Air [EMBRAER EMB14, EMBRAER EMB13]
Mesa Airlines Inc. [BOMBARDIER INC CL60, DEHAVILLAND DHC82, CANADAIR CL60]
Northwest Airlines Inc. [AIRBUS INDUSTRIE A32, BOEING 757, MCDONNELL DOUGLAS DC95, DOUGLAS DC93, AIRBUS A31]
PSA Airlines Inc. [BOMBARDIER INC CL60, CANADAIR CL60, PIPER PA28]
Pinnacle Airlines Inc. [BOMBARDIER INC CL60]
Skywest Airlines Inc. [BOMBARDIER INC CL60, EMBRAER EMB12, CANADAIR CL60]
Southwest Airlines Co. [BOEING 737, BOEING 747, EMBRAER EMB12, CESSNA T33, BEECH 65A90]
US Airways [BOEING 737, AIRBUS INDUSTRIE A31, AIRBUS INDUSTRIE A32, EMBRAER ERJ 19, BOEING 757]
United Airlines [BOEING 737, AIRBUS INDUSTRIE A32, BOEING 757, AIRBUS INDUSTRIE A31, AIRBUS A32]

In [0]:
stagemetrics.print_report()

Scheduling mode = FAIR
Spark Context default degree of parallelism = 8
Aggregated Spark stage metrics:
numStages => 16
numTasks => 20
elapsedTime => 9203 (9 s)
stageDuration => 5785 (6 s)
executorRunTime => 8398 (8 s)
executorCpuTime => 2571 (3 s)
executorDeserializeTime => 1187 (1 s)
executorDeserializeCpuTime => 592 (0.6 s)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 66 (66 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 275 (0.3 s)
resultSize => 43326 (42.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 102285312
recordsRead => 183011
bytesRead => 6460248 (6.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 814
shuffleTotalBlocksFetched => 9
shuffleLocalBlocksFetched => 9
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 57197 (55.0 KB)
shuffleLocalBytesRead => 57197 (55.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 47625 (46.0 KB)
shuffleRecordsWritten => 599